**YARISMA HAKKINDA ONEMLI**

Puanlama kriterleri, 5 skorun ortalaması:

> Türk Eğitim Sistemi, Türk Hukuku, Sürdürülebilirlik, Tarım ve veri setimizin kalitesi


- Yenilikçi yaklaşım ve Türkçe LLM ekosistemine katkı %20
- Veri setinin büyüklüğü, temizlenmesi ve kalitesi %30
- Model başarısı %50

**Güzel Öğrenilmesi Gereken Şeyler**

> [NbDev - Notebook üzerinden web app geliştirme toolkit](https://www.youtube.com/watch?v=l7zS8Ld4_iA)

> [LoRA,QLoRA methodlari hakkinda](https://www.youtube.com/watch?v=l5a_uKnbEr4)



In [ ]:
from datasets import load_dataset

ds = load_dataset("kadirnar/combined-turkish-datasets-v5")


In [ ]:
import pandas as pd

# Convert each split to a DataFrame
train_df = pd.DataFrame(ds['train'])
# Repeat for other splits if needed:
# test_df = pd.DataFrame(ds['test'])

In [39]:
train_df[train_df['instruction'].str.contains('yasa')]

,input,instruction,output
17,,Bir tasarının Amerika Birleşik Devletleri Kong...,1. Giriş: Bir Kongre üyesi tasarıyı Temsilcile...
24,,Amerika Birleşik Devletleri'nde bir yasa tasar...,1) Giriş: Bir Kongre üyesi tarafından Temsilci...
151,"IPhone 2007'de piyasaya sürüldü, dünyayı cihaz...",Bu metindeki dilbilgisi ve noktalama işaretler...,"iPhone 2007'de piyasaya sürüldü; Steve Jobs, c..."
201,,Newton'un hareket yasalarının kısa bir özetini...,Newton'un hareket yasaları nesnelerin hareketi...
222,C3H8 + 5O2 → 3CO2 + 4H2O,Kimyasal reaksiyonun türünü tanımlayın ve reak...,Bu bir yanma reaksiyonudur. Reaktifler: propan...
...,...,...,...
465531,,Küresel ekonomi ve finansal piyasaların işleyi...,"Küresel ekonomi, tüm dünya ülkelerinin birbirl..."
465613,,Türkiye ekonomisinin global piyasalara entegra...,Türkiye ekonomisinin global piyasalara entegra...
466211,,Türkiye ekonomisinin global piyasada en önemli...,"1. Tarım ve Hayvancılık: Türkiye, tarım ve hay..."
466530,,Küresel ekonomi ve finansal piyasalar konusund...,"2008 küresel finansal kriz, son yılların en ön..."


In [51]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string
from nltk.tokenize import word_tokenize
nltk.download('punkt')

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('turkish'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

train_df['processed_text'] = train_df['instruction'][0:500].apply(preprocess_text)

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_df['processed_text'][0:500])

num_topics = 10
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    top_features_ind = topic.argsort()[:-10 - 1:-1]
    top_features = [feature_names[i] for i in top_features_ind]
    print(f"Topic {topic_idx}: {', '.join(top_features)}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Topic 0: adım, açıklayın, bir, sürecini, temel, aşağıdaki, yapma, basit, verilen, olarak
Topic 1: bir, aşağıdaki, kısa, verilen, yapın, tanımlayın, mantıksal, açıklama, sıraya, kronolojik
Topic 2: bir, ana, açıklayın, aşağıdaki, özetleyin, verilen, alaycı, verin, kısa, kuşu
Topic 3: bir, aşağıdaki, çevirin, cümleyi, yazın, fransızcaya, yeniden, ngilizce, açıklayın, kısa
Topic 4: bir, açıklayın, edin, arasındaki, aşağıdaki, tanımlayın, temel, analiz, farkı, mantıksal
Topic 5: bir, açıklayın, aşağıdaki, yazın, edin, yeniden, verilen, şekilde, kısa, olduğunuzu
Topic 6: bir, kısa, yazın, açıklayın, üç, olarak, edin, seçmeli, çoktan, verilen
Topic 7: bir, açıklayın, verilen, analiz, aşağıdaki, tanımlayın, sınıflandırın, olarak, edin, üç
Topic 8: açıklayın, verilen, sürecini, tanımlayın, mantıksal, belirleyin, doğal, jane, seçilim, bunun
Topic 9: bir, verilen, aşağıdaki, tanımlayın, önerin, dilbilgisi, belirleyin, hatalarını, önemli, yeni


In [52]:
!pip install transformers

In [53]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [56]:
konular = ["Türk Eğitim Sistemi", "Türk Hukuku", "Sürdürülebilirlik", "Tarım"]
counter = 0
def siniflandir(metin):
    sonuc = classifier(metin, konular)
    en_yuksek_etiket = sonuc['labels'][0]
    en_yuksek_skor = sonuc['scores'][0]

    if en_yuksek_skor < 0.5:  # Eşik değeri belirleyin
        return "diğer"
    else:
        return en_yuksek_etiket

# DataFrame sütununu label'layın
train_df['etiket'] = train_df['instruction'][0:100].apply(siniflandir)